### Implementación de los métodos iterativos

In [37]:
"""
Implementacion del método de Richardson
"""

import numpy as np


def richardson(A, b, x0, MAX_ITERATIONS=1000):
    for k in range(MAX_ITERATIONS):
        r = b-np.dot(A, x0)
        x = x0 + r
        x0 = x
    
    return x


In [33]:
"""
Implementacion del método de Jacobi
"""

import numpy as np

def isdiag_domin(A):
    nrow = np.shape(A)[0]
    index = np.arange(nrow)
    for k in range(nrow):
        if abs(A[k,k]) < sum(np.abs(A[k, index[index != k]])):
            return False

    return True

def jacobi(A, b, x0, MAX_ITERATIONS=1000):
    if isdiag_domin(A):
        nrow = np.shape(A)[0]
        Q = np.diag(np.diag(A))
        x = np.zeros_like(x0)
        for k in range(MAX_ITERATIONS):
            b0 = np.dot(Q-A, x0)+b
            x0 = np.array([b0[k]/Q[k,k] for k in range(nrow)]) 
        return x

    else:
        print("El metodo de Jacobi no es adecuado.")
        return x0


In [34]:
"""
Implementacion del método de Gauss_Seidel
"""


import numpy as np  
import scipy.linalg as sl

def gauss_seidel(A, b, x0, MAX_ITERATIONS=1000):
    if isdiag_domin(A):
        Q = trinf(A)
        invQ = sl.inv(Q)
        nrow = np.shape(A)[0]
        G = np.identity(nrow)-np.dot(invQ, A)
        c = np.dot(invQ, b)
        for k in range(MAX_ITERATIONS):
            x = np.dot(G, x0) + c
            x0 = x
        return x

    else:
        print("El metodo de Gauss-Seidel no es adecuado.")
        return np.zeros_like(x0)
    
def isdiag_domin(A):
    nrow = np.shape(A)[0]
    index = np.arange(nrow)
    for k in range(nrow):
        if abs(A[k,k]) < sum(np.abs(A[k, index[index != k]])):
            return False
    return True



def trinf(A):
    Q = np.zeros_like(A)
    nrow = np.shape(A)[0]
    for k in range(nrow):
        Q[k,:k] = A[k, :k]

    return Q

#### Aplicación de los métodos iterativos a la solución de un sistema de ecuaciones 

In [35]:
"""
genera un sistema de cualquier orden (I-A)x = b donde Aij in <0,1>  , L < x < U y b > 0
"""


import numpy as np
import random as rdm

def vector_random(n, a, b):
    return np.array([rdm.uniform(a, b) for k in range(n)])

def genera_sistema(nrow, L, U):
    x = vector_random(nrow, L, U)
    A =  np.zeros((nrow, nrow))
    index = np.arange(nrow)
    for k in range(nrow):
        sk = sum(x[index != k])
        A[k, index[index != k]] = vector_random(nrow-1, 0, 1)/sk
        a = 1-1/x[k]
        if a > 0:
            A[k,k] = rdm.uniform(0, a)
        else:
            return np.zeros_like(A), np.zeros_like(x)
    
    return A, np.dot(A, x).reshape(nrow,1)

def busca_sistema(nrow, L, U, MAX_ITERATIONS=1000):
    for k in range(MAX_ITERATIONS):
        A, x = genera_sistema(nrow, L , U)
        A0 = np.identity(nrow)-A
        if np.alltrue(np.dot(A0,x) > 0):
            return A, x

    return np.zeros_like(A), np.zeros_like(x)

#### Resolución del sistema de orden 10 generado por la función *genera_sistema*

In [38]:
"""
rx = solucion debido a las iteraciones de richardson
rj = solucion debido a las iteraciones de jacobi
rgs = solucion debido a las iteraciones de gauss-seidel
rsor = solucion debido a las iteraciones de SOR

x0: punto de inicio de las iteraciones 
"""
import scipy.linalg as sl

A, b = genera_sistema(10, 1, 20)
x0 = np.zeros((10,1))
print("Soluciones del sistema:\nA:\n{}\n\nb:\n{}\n\nCon 1000 iteraciones".format(A, b))

# Solucion usando modulo sciy.linalg
x = sl.solve(A, b)
print("--------------------\nSolucion scipy.linalg:\n", x)

# Solucion por las iteraciones de richardson
rx = richardson(A, b, x0)
print("--------------------\nSolucion Richardson:\n", rx)

# Solucion por las iteraciones de jacobi
jx = jacobi(A, b, x0)
print("--------------------\nSolucion Jacobi:\n", jx)

# Solucion por las iteraciones de gauss_seidel

gsx = gauss_seidel(A, b, x0)
print("--------------------\nSolucion Gauss-Seidel:\n", gsx)

# Solucion por las iteraciones de SOR
# FALTA IMPLEMENTA METODO SE SOR
#print("Solucion SOR: {}")



Soluciones del sistema:
A:
[[5.26589940e-01 6.02086943e-03 2.85580366e-03 5.24124912e-03
  1.16952310e-03 7.51026148e-03 3.17439223e-03 3.92212371e-03
  1.43317079e-03 8.83929150e-05]
 [2.73697859e-03 6.22824040e-01 6.50537077e-03 4.67716058e-03
  3.17029973e-03 5.36280828e-03 7.10201601e-03 5.09832677e-03
  4.55283043e-03 3.99697987e-03]
 [7.71278892e-03 5.13140886e-04 7.80885895e-01 5.06889328e-03
  3.63194985e-03 1.53653048e-03 3.28062796e-03 1.34273824e-03
  4.19742531e-03 4.86401200e-03]
 [1.02525430e-03 8.00384902e-03 2.11468319e-03 3.86583317e-01
  7.12385744e-03 1.50466019e-03 5.77613611e-03 9.26582283e-04
  1.65068577e-03 1.61855619e-03]
 [1.74278522e-03 5.07592917e-03 4.30487613e-03 7.39332493e-03
  1.82708762e-01 6.81321156e-03 7.34290275e-03 6.08406006e-03
  3.81226035e-04 3.42975611e-04]
 [7.44613204e-03 1.00152045e-03 4.45232333e-03 6.56600591e-03
  4.11676899e-03 1.13862891e-01 3.02345883e-03 1.53178289e-03
  2.28530937e-03 1.42621185e-03]
 [6.51728492e-03 7.16687492e-03

LinAlgError: singular matrix